In [1]:
from plotting import CandlePlot
import datetime as dt
import plotly.graph_objects as go
import pandas as pd
import sys
sys.path.append("../")

In [2]:
from infrastructure.instrument_collection import instrumentCollection as ic

In [3]:
pair = "EUR_USD"
granularity = "H4"
df = pd.read_csv(f"../data/candles/{pair}_{granularity}.csv")
MA_LIST = [10, 20, 50, 100, 200]

In [4]:
df.columns

Index(['Unnamed: 0', 'time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
       'bid_o', 'bid_h', 'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c'],
      dtype='object')

In [5]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [6]:
df_ma.head(10)

,time,mid_o,mid_h,mid_l,mid_c
0,2013-01-06 22:00:00+00:00,1.30760,1.30794,1.30427,1.30532
1,2013-01-07 02:00:00+00:00,1.30533,1.30663,1.30288,1.30362
2,2013-01-07 06:00:00+00:00,1.30360,1.30549,1.30166,1.30372
3,2013-01-07 10:00:00+00:00,1.30372,1.30656,1.30222,1.30604
4,2013-01-07 14:00:00+00:00,1.30604,1.31132,1.30527,1.31081
5,2013-01-07 18:00:00+00:00,1.31082,1.31196,1.31044,1.31164
6,2013-01-07 22:00:00+00:00,1.31163,1.31282,1.31044,1.31118
7,2013-01-08 02:00:00+00:00,1.31118,1.31400,1.31037,1.31238
8,2013-01-08 06:00:00+00:00,1.31239,1.31325,1.31031,1.31170
9,2013-01-08 10:00:00+00:00,1.31172,1.31322,1.30692,1.30820


In [7]:
for ma in MA_LIST:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)

In [8]:
df_plot = df_ma.iloc[:500]

In [9]:
cp = CandlePlot(df_plot)

In [10]:
traces = [f"MA_{x}" for x in MA_LIST]

In [11]:
cp.show_plot(line_traces=traces)

In [12]:
MA_S = "MA_50"
MA_L = "MA_200"
BUY = 1
SELL = -1
NONE = 0

In [13]:
df_an = df_ma[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', MA_S, MA_L]].copy()

In [14]:
df_an.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200
0,2013-02-19 14:00:00+00:00,1.33461,1.33942,1.33452,1.33868,1.338628,1.337668
1,2013-02-19 18:00:00+00:00,1.33868,1.33960,1.33824,1.33878,1.338608,1.337835
2,2013-02-19 22:00:00+00:00,1.33877,1.34342,1.33816,1.34199,1.338638,1.338027
3,2013-02-20 02:00:00+00:00,1.34199,1.34218,1.34062,1.34142,1.338665,1.338216
4,2013-02-20 06:00:00+00:00,1.34142,1.34322,1.34084,1.34192,1.338668,1.338395


In [15]:
df_an['DELTA'] = df_an.MA_50 - df_an.MA_200
df_an['DELTA_PREV'] = df_an.DELTA.shift(1)

In [16]:
def is_trade(row):
    if row.DELTA >= 0 and row.DELTA_PREV < 0:
        return BUY
    if row.DELTA < 0 and row.DELTA_PREV >= 0:
        return SELL
    return 0

In [17]:
df_an['TRADE'] = df_an.apply(is_trade, axis=1)

In [18]:
df_trades = df_an[df_an.TRADE != NONE].copy()

In [19]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE
7,2013-02-20 18:00:00+00:00,1.33437,1.33584,1.32710,1.32825,1.338481,1.338729,-0.000248,0.000003,-1
229,2013-04-12 01:00:00+00:00,1.31107,1.31266,1.31082,1.31172,1.296941,1.296800,0.000141,-0.000375,1
374,2013-05-16 01:00:00+00:00,1.28798,1.28824,1.28664,1.28733,1.303203,1.303281,-0.000078,0.000445,-1
467,2013-06-06 13:00:00+00:00,1.31508,1.33064,1.31330,1.32506,1.300808,1.300721,0.000087,-0.000428,1
574,2013-07-02 09:00:00+00:00,1.30223,1.30340,1.29922,1.30024,1.308653,1.308821,-0.000168,0.000398,-1


In [20]:
cp = CandlePlot(df_an.iloc[15:55])
cp.show_plot(line_traces=[MA_S, MA_L])

In [21]:
ic.LoadInstruments("../data")

In [22]:
ic.instruments_dict[pair]

{'name': 'EUR_USD', 'ins_type': '', 'displayName': 'EUR/USD', 'pipLocation': 0.0001, 'tradeUnitsPrecision': 0, 'marginRate': 0.0333333}

In [23]:
ins_data = ic.instruments_dict[pair]

In [24]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE
7,2013-02-20 18:00:00+00:00,1.33437,1.33584,1.32710,1.32825,1.338481,1.338729,-0.000248,0.000003,-1
229,2013-04-12 01:00:00+00:00,1.31107,1.31266,1.31082,1.31172,1.296941,1.296800,0.000141,-0.000375,1
374,2013-05-16 01:00:00+00:00,1.28798,1.28824,1.28664,1.28733,1.303203,1.303281,-0.000078,0.000445,-1
467,2013-06-06 13:00:00+00:00,1.31508,1.33064,1.31330,1.32506,1.300808,1.300721,0.000087,-0.000428,1
574,2013-07-02 09:00:00+00:00,1.30223,1.30340,1.29922,1.30024,1.308653,1.308821,-0.000168,0.000398,-1


In [25]:
df_trades['DIFF'] = df_trades.mid_c.diff().shift(-1)
df_trades.fillna(0, inplace=True)

In [26]:
df_trades['GAIN'] = df_trades['DIFF'] / ins_data.pipLocation
df_trades['GAIN'] = df_trades['GAIN'] * df_trades['TRADE']

In [27]:
df_trades.GAIN.sum()

-785.2999999999975

In [28]:
df_trades['GAIN_C'] = df_trades['GAIN'].cumsum()

In [29]:
df_trades.shape

(110, 13)

In [30]:
cp = CandlePlot(df_trades, candles=False)
cp.show_plot(line_traces=['GAIN_C'])